In [ ]:
 !pip install --quiet langchain pydantic unstructured nest_asyncio openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 76.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


In [ ]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import json
from langchain.schema.document import Document
with open('cleaned_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
docs = [Document(page_content=doc_dict['page_content'], metadata=doc_dict['metadata']) for doc_dict in data]


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

docs_Chunks = text_splitter.split_documents(docs)

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings

In [ ]:
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {'device': 'cpu'}
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
!pip install --quiet pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 34.2 MB/s eta 0:00:00


In [ ]:
import pinecone

# initialize pinecone
pinecone.init(
    api_key="",
    environment="gcp-starter"
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
index = pinecone.Index("bullbot")
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
for idx, doc in enumerate(docs_Chunks):
    metadata_size = len(str(docs_Chunks[idx].page_content).encode('utf-8'))
    if metadata_size > 30500:  # 40960 bytes is the limit
        print(f"Document at index {idx} has metadata size: {metadata_size} bytes")
        docs_Chunks.pop(idx)
        print(f"Removed document at index {idx}")
len(docs_Chunks)

3554

In [ ]:
from langchain.vectorstores import Pinecone

index_name = "bullbot"

# create a new index
docsearch = Pinecone.from_documents(docs_Chunks, embeddings, index_name = index_name)


# existing index
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [ ]:
index = pinecone.Index("bullbot")
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.035,
 'namespaces': {'': {'vector_count': 3500}},
 'total_vector_count': 3500}